In [1]:
import numpy as np # linear algebra
import os
import json
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
tf.config.set_visible_devices([], 'CPU') # hide the CPU
tf.config.set_visible_devices(gpus[0], 'GPU') # unhide potentially hidden GPU
tf.config.get_visible_devices()

IMAGE_SIZE = [64, 64]

train_path = '/content/drive/MyDrive/ RCNN/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train'
valid_path = '/content/drive/MyDrive/ RCNN/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test'

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False

folders = glob('/content/drive/MyDrive/ RCNN/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train')

x = Flatten()(vgg.output)

prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/ RCNN/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train',
                                                 target_size = (64 , 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ RCNN/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test',
                                            target_size = (64 , 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=16,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

test_images = []

for img in os.listdir(valid_path):
    test_image = plt.imread(os.path.join(valid_path,img))
    test_images.append(test_image.reshape([64,64,3]))
test_images = np.array(test_images)

final_preds=[]
model_predictions = model.predict(test_images)
for i in model_predictions:
    if (i >= 1).all:
        final_preds.append(1)
    if (i < 1).all:
        final_preds.append(0)

dictio={}
for i,j in enumerate(os.listdir(valid_path)):
    dictio[j] = final_preds[i]

def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

def generate_sample_file(filename):
    write_json(filename, dictio)


generate_sample_file('./hackathonresult.json')

Mounted at /content/drive
[]


IndexError: list index out of range

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
